In [1]:
"""
Baseline for machine learning project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss

Credits: Aurelien Lucchi, ETH Zürich
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image
import code
import tensorflow.python.platform
import numpy
import tensorflow as tf

from tensorflow.keras import backend as K
import gc

NUM_CHANNELS = 3  # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2

TOTAL_DATA_SIZE=100
DATA_IDS = numpy.array([i for i in range(1,TOTAL_DATA_SIZE+1) if i != 33])
TRAIN_SIZE = 89
VALIDATE_SIZE = 10
ROTATION = True

GROUPED_BATCH_SIZE = 3

numpy.random.seed(42)
IDS = numpy.random.choice(DATA_IDS, size=(TRAIN_SIZE+VALIDATE_SIZE), replace=False, p=None)
TRAIN_IDS = IDS[:TRAIN_SIZE]
VALIDATE_IDS = IDS[TRAIN_SIZE:]
if ROTATION:
    TRAIN_IDS = numpy.array([j for i in TRAIN_IDS for j in range(i, TOTAL_DATA_SIZE*8+1, TOTAL_DATA_SIZE) ])
    VALIDATE_IDS = numpy.array([j for i in VALIDATE_IDS for j in range(i, TOTAL_DATA_SIZE*8+1, TOTAL_DATA_SIZE) ])


#VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 16  # 64
NUM_EPOCHS = 100
RESTORE_MODEL = False  # If True, restore existing model instead of training a new one
RECORDING_STEP = 0

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

tf.app.flags.DEFINE_string('train_dir', '/tmp/segment_aerial_images',
                           """Directory where to write event logs """
                           """and checkpoint.""")
FLAGS = tf.app.flags.FLAGS

In [2]:
TRAIN_IDS.shape, VALIDATE_IDS.shape, TRAIN_IDS[:3], VALIDATE_IDS[:3]

((712,), (80,), array([ 64, 164, 264]), array([ 92, 192, 292]))

In [3]:
# Extract patches from a given image
def img_crop(im, w, h):
    h_shift = (GROUPED_BATCH_SIZE // 2)*h
    w_shift = (GROUPED_BATCH_SIZE // 2)*w
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(h_shift, imgheight - h_shift, h):
        for j in range(w_shift, imgwidth - w_shift, w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[
                    j-w_shift:j+w+w_shift,
                    i-h_shift:i+h+h_shift,
                    :]
            list_patches.append(im_patch)
    return list_patches

In [4]:
def extract_data(filename, image_ids):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in image_ids:
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return numpy.asarray(data)

In [5]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.25  # percentage of pixels > 1 required to assign a foreground label to a patch
    df = numpy.sum(v)
    if df > foreground_threshold:  # road
        return [0, 1]
    else:  # bgrd
        return [1, 0]

In [6]:
# Extract label images
def extract_labels(filename, image_ids):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in image_ids:
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = numpy.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = numpy.asarray([value_to_class(numpy.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(numpy.float32)

In [7]:
def error_rate(predictions, labels):
    """Return the error rate based on dense predictions and 1-hot labels."""
    return 100.0 - (
        100.0 *
        numpy.sum(numpy.argmax(predictions, 1) == numpy.argmax(labels, 1)) /
        predictions.shape[0])

In [8]:
# Write predictions from neural network to a file
def write_predictions_to_file(predictions, labels, filename):
    max_labels = numpy.argmax(labels, 1)
    max_predictions = numpy.argmax(predictions, 1)
    file = open(filename, "w")
    n = predictions.shape[0]
    for i in range(0, n):
        file.write(max_labels(i) + ' ' + max_predictions(i))
    file.close()

In [9]:
# Print predictions from neural network
def print_predictions(predictions, labels):
    max_labels = numpy.argmax(labels, 1)
    max_predictions = numpy.argmax(predictions, 1)
    print(str(max_labels) + ' ' + str(max_predictions))

In [10]:
# Convert array of labels to an image
def label_to_img(imgwidth, imgheight, w, h, labels):
    array_labels = numpy.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0, imgheight, h):
        for j in range(0, imgwidth, w):
            if labels[idx][0] > 0.5:  # bgrd
                l = 0
            else:
                l = 1
            array_labels[j:j+w, i:i+h] = l
            idx = idx + 1
    return array_labels

In [11]:
def img_float_to_uint8(img):
    rimg = img - numpy.min(img)
    rimg = (rimg / numpy.max(rimg) * PIXEL_DEPTH).round().astype(numpy.uint8)
    return rimg

In [12]:
def concatenate_images(img, gt_img):
    n_channels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if n_channels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:, :, 0] = gt_img8
        gt_img_3c[:, :, 1] = gt_img8
        gt_img_3c[:, :, 2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg

In [13]:
def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = numpy.zeros((w, h, 3), dtype=numpy.uint8)
    color_mask[:, :, 0] = predicted_img*PIXEL_DEPTH

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

In [14]:
def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos
    
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return numerator / (denominator + K.epsilon())

def bcor(y_true, y_pred):
    pp = K.mean(K.round(K.clip(y_pred, 0, 1)))
    pn = 1 - pp
    pos = K.mean(K.round(K.clip(y_true, 0, 1)))
    neg = 1 - pos
    
    tp = K.mean(K.round(K.clip(y_true * y_pred, 0, 1)))
    fp = pp - tp
    
    fn = pos - tp
    tn = pn - fn
    
    return (tp - (pp*pos)) / (pos - (pos*pos))

def precision(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    return precision

def recall(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pos + K.epsilon())
    return recall

def f1(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    pp = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = tp / (pp + K.epsilon())
    
    pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = tp / (pos + K.epsilon())
    
    return 2*((precision * recall) / (precision + recall + K.epsilon()))

In [15]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/'
test_data_filename = 'test_set_images/'

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAIN_IDS)
validate_data = extract_data(train_data_filename, VALIDATE_IDS)

train_labels = extract_labels(train_labels_filename, TRAIN_IDS)
validate_labels = extract_labels(train_labels_filename, VALIDATE_IDS)

In [16]:
num_epochs = NUM_EPOCHS

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class before balancing: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(train_labels) if j[0] == 1]
idx1 = [i for i, j in enumerate(train_labels) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(train_data.shape)
train_data = train_data[new_indices, :, :, :]
train_labels = train_labels[new_indices]

train_size = train_labels.shape[0]

c0 = 0
c1 = 0
for i in range(len(train_labels)):
    if train_labels[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class after balancing: c0 = ' + str(c0) + ' c1 = ' + str(c1))
print(len(new_indices))
print(train_data.shape)

Number of data points per class before balancing: c0 = 277680 c1 = 98968
Balancing training data...
197936
(376648, 48, 48, 3)
Number of data points per class after balancing: c0 = 98968 c1 = 98968
197936
(197936, 48, 48, 3)


In [48]:
#tn = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=SEED)
#const = tf.keras.initializers.Constant(value=0.1)
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (6, 6), strides=2,
                         activation=tf.nn.relu
                        ),
  tf.keras.layers.Conv2D(64, (4, 4), strides=2,
                         activation=tf.nn.relu
                        ),
  tf.keras.layers.Conv2D(128, (5, 5),
                         activation=tf.nn.relu
                        ),
  tf.keras.layers.Conv2D(512, (5, 5),
                         activation=tf.nn.relu
                        ),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512,
                         activation=tf.nn.relu
                       ),
  tf.keras.layers.Dense(2,
                         activation=None
                       ),
])

In [49]:
gc.collect()

0

In [50]:
# Compile model
optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy', f1, precision, recall, mcor, bcor])

In [51]:
RESTORE_MODEL

False

In [ ]:
# Train
if not RESTORE_MODEL:
    model.fit(train_data, train_labels, batch_size=128, epochs=10)
    model.save_weights("./weigths.hdf5")
else:
    model.load_weights("./weigths.hdf5")

Epoch 1/10
197936/197936 [==============================] - 28s 142us/step - loss: 0.6794 - acc: 0.5643 - f1: 0.6661 - precision: 0.4997 - recall: 0.9990 - mcor: 2.0799e-04 - bcor: 5.0521e-05
Epoch 2/10
197936/197936 [==============================] - 27s 138us/step - loss: 0.6471 - acc: 0.6262 - f1: 0.6667 - precision: 0.5000 - recall: 1.0000 - mcor: 0.0014 - bcor: 1.9703e-04
Epoch 3/10
197936/197936 [==============================] - 28s 139us/step - loss: 0.6117 - acc: 0.6668 - f1: 0.6668 - precision: 0.5002 - recall: 0.9999 - mcor: 0.0040 - bcor: 7.0225e-04
Epoch 4/10
197936/197936 [==============================] - 28s 139us/step - loss: 0.5712 - acc: 0.6893 - f1: 0.6689 - precision: 0.5026 - recall: 0.9998 - mcor: 0.0457 - bcor: 0.0101
Epoch 5/10
197936/197936 [==============================] - 28s 139us/step - loss: 0.6602 - acc: 0.5996 - f1: 0.6667 - precision: 0.5000 - recall: 1.0000 - mcor: 0.0000e+00 - bcor: 0.0000e+00
Epoch 6/10
197936/197936 [==============================

In [ ]:
model.summary()

In [ ]:
model.evaluate(validate_data, validate_labels)